In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.python.framework import dtypes
from solarimage.classifier.nn.trainer import TrainIter

In [3]:
mnist = input_data.read_data_sets("example/MNIST_data/", one_hot=True)

Extracting example/MNIST_data/train-images-idx3-ubyte.gz


Extracting example/MNIST_data/train-labels-idx1-ubyte.gz
Extracting example/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting example/MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
trainer = TrainIter(32, 1000, 
                    images=mnist.train.images, 
                    labels=mnist.train.labels, 
                    sample_size=64,
                    hidden_size=128,
                    learning_rate=1e-3,
                    reshape=False,
                    dtype=dtypes.uint8)

In [5]:
d_loss_list, g_loss_list = trainer.run_genradvers()
# z, theta_g, x, theta_d = trainer.get_genradvers_var()
# d_loss, g_loss = trainer.get_genradvers_loss(z, theta_g, x, theta_d, trainer.batch_size)
# d_solver, g_solver = trainer.get_genradvers_solver(d_loss, theta_d, g_loss, theta_g)

In [6]:
d_loss_list

[3.9979839,
 3.5998471,
 3.5669134,
 3.5909932,
 3.533967,
 3.532711,
 3.5099607,
 3.5080726,
 3.5577197,
 3.5515223,
 3.5067558,
 3.5397043,
 3.5213306,
 3.5340202,
 3.52196,
 3.5265362,
 3.5209658,
 3.5088415,
 3.5027642,
 3.5010417,
 3.4866295,
 3.5077255,
 3.4960368,
 3.4999113,
 3.4947238,
 3.4999657,
 3.4951181,
 3.4938231,
 3.5074492,
 3.4929857,
 3.49224,
 3.5023463,
 3.5139096,
 3.5018315,
 3.5053039,
 3.5030847,
 3.4939525,
 3.4885604,
 3.4912405,
 3.4873075,
 3.4888623,
 3.4861467,
 3.4874027,
 3.5020819,
 3.4857445,
 3.4970298,
 3.4923105,
 3.4867666,
 3.4845264,
 3.4852514,
 3.4837117,
 3.4783864,
 3.4794867,
 3.4840817,
 3.4887333,
 3.4808717,
 3.4813383,
 3.4847317,
 3.4905162,
 3.4945791,
 3.4857965,
 3.4846985,
 3.4836903,
 3.4831159,
 3.4859974,
 3.4947174,
 3.4826601,
 3.4864106,
 3.485477,
 3.4893277,
 3.4814088,
 3.4872611,
 3.4840991,
 3.489032,
 3.4828212,
 3.4862323,
 3.4812484,
 3.4867351,
 3.4798014,
 3.4823449,
 3.4857626,
 3.4861178,
 3.4925189,
 3.4866865,


In [16]:
mb_size = 32
x_dim = 784
z_dim = 100
h_dim = 128
lr = 1e-3
d_steps = 3

In [17]:
# Discriminator Net
X = tf.placeholder(tf.float32, shape=[None, X_dim], name='X')

D_W1 = tf.Variable(xavier_init([x_dim, h_dim]), name='D_W1')
D_b1 = tf.Variable(tf.zeros(shape=[h_dim]), name='D_b1')
D_W2 = tf.Variable(xavier_init([h_dim, 1]), name='D_W2')
D_b2 = tf.Variable(tf.zeros(shape=[1]), name='D_b2')

theta_D = [D_W1, D_W2, D_b1, D_b2]

# Generator Net
Z = tf.placeholder(tf.float32, shape=[None, z_dim], name='Z')

G_W1 = tf.Variable(xavier_init([z_dim, h_dim]), name='G_W1')
G_b1 = tf.Variable(tf.zeros(shape=[h_dim]), name='G_b1')
G_W2 = tf.Variable(xavier_init([h_dim, x_dim]), name='G_W2')
G_b2 = tf.Variable(tf.zeros(shape=[x_dim]), name='G_b2')

theta_G = [G_W1, G_W2, G_b1, G_b2]

In [18]:
G_sample, _ = log_linear(Z, G_W1, G_b1, G_W2, G_b2)
D_real, D_logit_real = log_linear(X, D_W1, D_b1, D_W2, D_b2)
D_fake, D_logit_fake = log_linear(G_sample, D_W1, D_b1, D_W2, D_b2)
D_loss, G_loss = genradvers(D_real, D_fake)

In [19]:
# Only update D(X)'s parameters, so var_list = theta_D
D_solver = adam_optimizer(D_loss, theta_D)
# Only update G(X)'s parameters, so var_list = theta_G
G_solver = adam_optimizer(G_loss, theta_G)

In [27]:
def sample_Z(m, n):
    '''Uniform prior for G(Z)'''
    return np.random.uniform(-1., 1., size=[m, n])

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for _ in range(1000000):
        #  train process
        x_mb, batch_ys = train_dataset.next_batch(mb_size, shuffle=False)
        _, D_loss_curr = sess.run([D_solver, D_loss], feed_dict={X: x_mb, Z: sample_Z(mb_size, z_dim)})
        _, G_loss_curr = sess.run([G_solver, G_loss], feed_dict={Z: sample_Z(mb_size, z_dim)})